In [2]:
from qiskit import QuantumCircuit
from qiskit_aer import AerProvider
from qiskit.circuit import Clbit, ClassicalRegister
from qiskit_experiments.library import StateTomography
from qiskit.visualization import array_to_latex, plot_state_city
from qiskit.quantum_info import DensityMatrix, Pauli
from itertools import repeat, product
from operator import add, mul
import numpy as np
from qiskit.primitives import Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

In [5]:
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.draw()

┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

In [6]:
st = StateTomography(qc)
backend = Aer.get_backend('aer_simulator')
stdata = st.run(backend, shots=1000).block_for_results()
state_result = stdata.analysis_results("state")
# array_to_latex(state_result.value)
plot_state_city(state_result.value)


NameError: name 'Aer' is not defined

# Stoke's parameters tomography

In [28]:
def n_tomography(qc: QuantumCircuit, meas_qbits: list[int]) -> DensityMatrix:
    basis = [1, 2, 3]
    n_qbits = len(meas_qbits)
    p = np.zeros(list(repeat(6, n_qbits)))
    s = np.zeros(list(repeat(4, n_qbits)))
    s[0, 0] = 1
    for measurement in product(*repeat(basis, n_qbits)):
        print(f"Performing measurement {measurement}")
        sampler = Sampler()
        _qc = qc.copy()
        _qc.name = "".join("".join(str(i) for i in measurement))
        for i in range(0, n_qbits):
            m = measurement[i]
            if m == 1:
                _qc.h(meas_qbits[i])
            elif m == 2:
                _qc.sdg(meas_qbits[i])
                _qc.h(meas_qbits[i])
        
        cl_reg = ClassicalRegister(bits=[Clbit() for _ in range(n_qbits)])
        _qc.add_register(cl_reg)
        _qc.measure(meas_qbits, list(range(0, n_qbits)))   

        job = sampler.run(_qc)
        result = job.result()

        # print(f"Result: {result.quasi_dists[0]}")

        i = 0
        start = (np.array(measurement) - 1) * 2
        for delta in product(*repeat([0, 1], n_qbits)):
            res_obj = result.quasi_dists[0]
            prob = res_obj[i] if i in res_obj else 0
            pos = start + np.array(delta)
            p[*pos] = prob

            if sum(pos) % 2 == 0:
                s[*measurement] += prob
            else:
                s[*measurement] -= prob

            i += 1
    # print(p)
    for i in basis:
        j = 0
        start = (np.array([i, i]) - 1) * 2
        for delta in product(*repeat([0, 1], n_qbits)):
            pos = start + np.array(delta)
            if j % 2 == 0:
                s[0, i] += p[*pos]
            else:
                s[0, i] -= p[*pos]

            if j < (2 ** n_qbits) / 2:
                s[i, 0] += p[*pos]
            else:
                s[i, 0] -= p[*pos]

            j += 1
    # print(p)
    # print(s)

    p_map = ["I", "X", "Y", "Z"]
    density = np.zeros((2**n_qbits, 2**n_qbits), dtype=np.complex128)
    for measurement in product(*repeat([0] + basis, n_qbits)): 
        key = "".join([p_map[i] for i in measurement])
        pauli = Pauli(key).to_matrix()
        density += s[*measurement] * pauli
    density /= (2 ** n_qbits)

    return DensityMatrix(density)

qc = QuantumCircuit(2)
qc.h(0)
qc.h(1)

array_to_latex(n_tomography(qc, [0, 1]).data)


Performing measurement (1, 1)
Performing measurement (1, 2)
Performing measurement (1, 3)
Performing measurement (2, 1)
Performing measurement (2, 2)
Performing measurement (2, 3)
Performing measurement (3, 1)
Performing measurement (3, 2)
Performing measurement (3, 3)


<IPython.core.display.Latex object>

In [7]:
array_to_latex(DensityMatrix(qc).data)

<IPython.core.display.Latex object>

In [16]:
from qiskit.quantum_info import Pauli

array_to_latex(Pauli("XYX").to_matrix())

<IPython.core.display.Latex object>

In [4]:
from common.qiskit_run import create_density_matrix
from json import load

with open("./quantum/data/amplitude_damping.json") as f:
	exp = load(f)

jobs = exp[0]["jobs"]

dm = create_density_matrix(jobs)
array_to_latex(dm.data)

<IPython.core.display.Latex object>